In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('C:/Users/phpunsal/Documents/Data_Analytics/Flights/flights.db')

In [2]:
# initial sqlite_master query to identify what tables are in the database
query = """
SELECT name 
FROM sqlite_master 
WHERE type = 'table';
"""
df = pd.read_sql_query(query, conn)
df

,name
0,airports
1,airlines
2,routes


In [3]:
# this query gives a breakdown of the each table's columns and data types
for table in ['airports','airlines','routes']:
    
    query = f"""
    SELECT sql 
    FROM sqlite_master 
    WHERE name = '{table}';
    """
    
    df = pd.read_sql_query(query, conn)
    print(''.join(df.values[0, 0]))

CREATE TABLE airports (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [city] TEXT,
  [country] TEXT,
  [code] TEXT,
  [icao] TEXT,
  [latitude] TEXT,
  [longitude] TEXT,
  [altitude] TEXT,
  [offset] TEXT,
  [dst] TEXT,
  [timezone] TEXT
)
CREATE TABLE airlines (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [alias] TEXT,
  [iata] TEXT,
  [icao] TEXT,
  [callsign] TEXT,
  [country] TEXT,
  [active] TEXT
)
CREATE TABLE routes (
[index] INTEGER,
  [airline] TEXT,
  [airline_id] TEXT,
  [source] TEXT,
  [source_id] TEXT,
  [dest] TEXT,
  [dest_id] TEXT,
  [codeshare] TEXT,
  [stops] TEXT,
  [equipment] TEXT
)


In [4]:
# this query shows the contents of the airports table
query = """
SELECT *
FROM airports;
"""
df = pd.read_sql_query(query, conn)
df.head(2)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby


In [5]:
# this query shows the contents of the airlines table
query = """
SELECT *
FROM airlines;
"""
df = pd.read_sql_query(query, conn)
df.head(2)

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N


In [6]:
#this query shows the contents of the routes table
query = """
SELECT *
FROM routes;
"""
df = pd.read_sql_query(query, conn)
df.head(2)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2


In [7]:
# this query performs an inner join on the routes and airlines tables, connecting the airline_id/id columns
# and returns a table of the airline, airline_id and name
query = """
SELECT routes.airline, routes.airline_id, airlines.name
FROM routes 
INNER JOIN airlines 
ON routes.airline_id = airlines.id;
""" 
df = pd.read_sql_query(query, conn)
df

,airline,airline_id,name
0,2B,410,Aerocondor
1,2B,410,Aerocondor
2,2B,410,Aerocondor
3,2B,410,Aerocondor
4,2B,410,Aerocondor
...,...,...,...
66980,ZL,4178,Regional Express
66981,ZM,19016,Apache Air
66982,ZM,19016,Apache Air
66983,ZM,19016,Apache Air


In [8]:
# this query performs a left join from the routes table to the airlines table, connecting the airline_id/id columns
# and returns a similar table to the above query, but with approx. 700 more rows from the Routes table
query = """
SELECT routes.airline, routes.airline_id, airlines.name
FROM routes
LEFT JOIN airlines
ON routes.airline_id = airlines.id;
"""
df = pd.read_sql_query(query, conn)
df

,airline,airline_id,name
0,2B,410,Aerocondor
1,2B,410,Aerocondor
2,2B,410,Aerocondor
3,2B,410,Aerocondor
4,2B,410,Aerocondor
...,...,...,...
67658,ZL,4178,Regional Express
67659,ZM,19016,Apache Air
67660,ZM,19016,Apache Air
67661,ZM,19016,Apache Air


In [9]:
# this query performs an inner join on the routes and airports tables, connecting the source_id/id columns
# and returns a table of the origin of each route, the city of the origin airport and its longitude and latitude
query = '''
SELECT routes.source_id, airports.city, airports.latitude, airports.longitude
FROM routes
INNER JOIN airports
ON routes.source_id = airports.id
LIMIT 5;'''

df = pd.read_sql_query(query, conn)
df

,source_id,city,latitude,longitude
0,2965,Sochi,43.449928,39.956589
1,2966,Astrakhan,46.283333,48.006278
2,2966,Astrakhan,46.283333,48.006278
3,2968,Chelyabinsk,55.305836,61.503333
4,2968,Chelyabinsk,55.305836,61.503333


In [10]:
# this query performs an inner join twice on the routes and airports tables, connecting the source_id/id columns and 
# then the dest_id/id columns to return a table of both flight source and flight destination longitude and latitudes
query = """
SELECT sa.city AS source_city,
    sa.longitude AS origin_lon,
    sa.latitude AS origin_lat,
    da.longitude AS dest_lon,
    da.latitude AS dest_lat
FROM routes

INNER JOIN airports AS sa
ON sa.id = routes.source_id

INNER JOIN airports AS da
ON da.id = routes.dest_id

LIMIT 5;
"""
df = pd.read_sql_query(query, conn)
df

,source_city,origin_lon,origin_lat,dest_lon,dest_lat
0,Sochi,39.956589,43.449928,49.278728,55.606186
1,Astrakhan,48.006278,46.283333,49.278728,55.606186
2,Astrakhan,48.006278,46.283333,43.081889,44.225072
3,Chelyabinsk,61.503333,55.305836,49.278728,55.606186
4,Chelyabinsk,61.503333,55.305836,82.650656,55.012622


In [11]:
# this query performs an inner join of the routes and airlines tables, connecting the airline/id columns,
# to return a table of the airlines, their id, how many new routes they have, in descending order
query = '''
SELECT airlines.name, routes.airline_id, COUNT(*) AS new_routes
FROM routes
INNER JOIN airlines
ON routes.airline_id=airlines.id
GROUP BY routes.airline_id
ORDER BY new_routes DESC
LIMIT 5;
'''

df = pd.read_sql_query(query, conn)
df

,name,airline_id,new_routes
0,Ryanair,4296,2484
1,American Airlines,24,2354
2,United Airlines,5209,2180
3,Delta Air Lines,2009,1981
4,US Airways,5265,1960


In [12]:
# this query takes the name, city and country columns from the airports table and returns a 
# table of those columns, but with the airports ranked by timezone, then ordered by latitude
query = '''
SELECT name, city, country,
RANK() OVER(PARTITION BY airports.timezone ORDER BY airports.latitude ASC)
FROM airports;
'''

df = pd.read_sql_query(query, conn)
df

,name,city,country,RANK() OVER(PARTITION BY airports.timezone ORDER BY airports.latitude ASC)
0,San Pedro,San Pedro,Cote d'Ivoire,1
1,Abidjan Felix Houphouet Boigny Intl,Abidjan,Cote d'Ivoire,2
2,Daloa,Daloa,Cote d'Ivoire,3
3,Yamoussoukro,Yamoussoukro,Cote d'Ivoire,4
4,Man,Man,Cote d'Ivoire,5
...,...,...,...,...
8102,Layang Layang Airport,Layang Layang Atoll,Malaysia,88
8103,Uummannaq Heliport,Uummannaq,Greenland,89
8104,Dillant Hopkins Airport,Keene,United States,90
8105,Vilamendhoo,Vilamendhoo,Maldives,91


In [ ]:
# this query performs two inner joins, connecting source_id/id and then airline_id/id
query = """
SELECT DISTINCT al.name, ap.country,
COUNT(*) OVER(PARTITION BY al.name, ap.country ORDER BY al.name ) AS num_flights
FROM routes
INNER JOIN airports AS ap ON ap.id = routes.source_id
INNER JOIN airlines AS al ON al.id = routes.airline_id
LIMIT 50
"""
df = pd.read_sql_query(query, conn)
df.head(10)